In [2]:
import os, shutil
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from random import shuffle
from zipfile import ZipFile
from datetime import datetime
from google.colab import drive
from google.colab import files
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Understanding

In [ ]:
# Import the dataset
zipfile = 'gdrive/My Drive/shopee/shopee-product-detection-dataset-002.zip'
destination = 'dataset/'
drive.mount('/content/gdrive')

In [ ]:
# Unzip the dataset
before = datetime.now()
with ZipFile(zipfile, 'r') as zipObj:
    print("Unzipping")
    zipObj.extractall(destination)
after = datetime.now()
print('Done in', after-before)

In [5]:
# Specify path to the dataset
train_dir = 'dataset/train/train/'
validation_dir = 'dataset/validation/validation/'
test_dir = 'dataset/test/'

In [6]:
# Create directory for validation data
os.mkdir(os.path.dirname(validation_dir[:-1]))
os.mkdir(validation_dir)

In [7]:
# Split training and validation set
for category in os.listdir(train_dir):

    # Create new directory for each category
    os.mkdir(validation_dir + category)

    # Count number of images per category
    filenames = os.listdir(train_dir + category)
    total = len(filenames)
    fraction = int(total * 0.1)

    # Move files randomly to validation set
    shuffle(filenames)
    for filename in filenames[:fraction]:
        shutil.move(
            train_dir + category + '/' + filename, 
            validation_dir + category + '/' + filename
        )

In [8]:
# Function to count number of images
def count_images(location):
    counter = 0
    for path, subdirs, files in os.walk(location):
        for name in files:
            if name.endswith(".jpg"):
                counter = counter + 1
    
    return(counter)

In [ ]:
# Count number of images
total_train = count_images(train_dir)
total_validation = count_images(validation_dir)

print("Total training images:", total_train)
print("Total validation images:", total_validation)

In [10]:
# Specify hyperparameters
batch_size = 512
epochs = 5
IMG_HEIGHT = 150
IMG_WIDTH = 150

## Data Preparation

In [11]:
# Define image generator
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)
test_image_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
# Define flow from the training data
train_data_gen = train_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=train_dir,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical'
)

In [ ]:
# Define flow from the validation data
val_data_gen = validation_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=validation_dir,
    shuffle=False,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical'
)

In [ ]:
# Define flow from the test data
test_data_gen = test_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=test_dir,
    shuffle=False,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical'
)

## Modeling

In [ ]:
# Define model architecture
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(42)
])

In [ ]:
# Compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
# Show summary of the model
model.summary()

In [ ]:
# Train the model
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train//batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_validation//batch_size
)

In [64]:
# Make prediction for test data
predictions = np.argmax(model.predict(test_data_gen), axis=-1)

In [ ]:
# Get filenames of test data
filenames = test_data_gen.filenames

In [65]:
# Get list of test images for submission
test = pd.read_csv('dataset/test.csv')

In [66]:
# Generate submission file
submission=pd.DataFrame({"filename": filenames, "category": predictions})
submission.category = submission.category.astype('str')
submission['filename'] = submission.apply(lambda row: row[0][5:], axis=1)
submission['category'] = submission.apply(lambda row: row[1].zfill(2), axis=1)
submission = pd.merge(test[['filename']], submission, on='filename', how='left')
submission.to_csv('submission.csv', index=False, header=True)

In [ ]:
# Download submission file
files.download('submission.csv') 